## Домашнее задание к занятию "A/B-тесты"

### Описание задачи

![banner](https://storage.googleapis.com/kaggle-datasets-images/635/1204/126be74882028aac7241553cef0e27a7/dataset-original.jpg)

Покемоны - это маленькие существа, которые сражаются друг с другом на соревнованиях. Все покемоны имеют разные характеристики (сила атаки, защиты и т. д.) И относятся к одному или двум так называемым классам (вода, огонь и т. д.).
Профессор Оук является изобретателем Pokedex - портативного устройства, которое хранит информацию обо всех существующих покемонах. Как его ведущий специалист по данным, Вы только что получили от него запрос с просьбой осуществить аналитику данных на всех устройствах Pokedex.

### Описание набора данных
Профессор Оук скопировал все содержимое в память одного устройства Pokedex, в результате чего получился набор данных, с которым Вы будете работать в этой задаче. В этом файле каждая строка представляет характеристики одного покемона:

* `pid`: Numeric - ID покемона
* `HP`: Numeric - Очки здоровья
* `Attack`: Numeric - Сила обычной атаки
* `Defense`: Numeric - Сила обычной защиты
* `Sp. Atk`: Numeric - Сила специальной атаки
* `Sp. Def`: Numeric - Сила специальной защиты
* `Speed`: Numeric - Скорость движений
* `Legendary`: Boolean - «True», если покемон редкий
* `Class 1`: Categorical - Класс покемона
* `Class 2`: Categorical - Класс покемона

In [1]:
import scipy.stats as st
import numpy as np

In [2]:
import warnings
# Отключение предупреждений (warnings)
warnings.filterwarnings("ignore")

import pandas as pd

from scipy.stats import ttest_ind
from scipy.stats import f_oneway, shapiro

pokemon = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/datasets_for_t-tests/main/pokemon.csv', on_bad_lines='skip')  # Откроем датасет
pokemon.head()

# Обратите внимание, что у покемона может быть один или два класса.
# Если у покемона два класса, считается, что они имеют одинаковую значимость.

,pid,Name,Class 1,Class 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,False


In [3]:
pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   pid        800 non-null    int64 
 1   Name       799 non-null    object
 2   Class 1    800 non-null    object
 3   Class 2    414 non-null    object
 4   HP         800 non-null    int64 
 5   Attack     800 non-null    int64 
 6   Defense    800 non-null    int64 
 7   Sp. Atk    800 non-null    int64 
 8   Sp. Def    800 non-null    int64 
 9   Speed      800 non-null    int64 
 10  Legendary  800 non-null    bool  
dtypes: bool(1), int64(7), object(3)
memory usage: 63.4+ KB


### Задачи

<div class="alert alert-info">
<b>Задание № 1:</b>
    
Профессор Оук подозревает, что покемоны в классе `Grass` имеют более сильную обычную атаку, чем покемоны в классе `Rock`. Проверьте, прав ли он, и убедите его в своём выводе статистически.
    
    
Примечание: если есть покемоны, которые относятся к обоим классам, просто выбросьте их;
    
Вы можете предположить, что распределение обычных атак является нормальным для всех классов покемонов.

</div>

In [4]:
# Проверка на повторы
pokemon['pid'].unique

<bound method Series.unique of 0        1
1        2
2        3
3        4
4        5
      ... 
795    796
796    797
797    798
798    799
799    800
Name: pid, Length: 800, dtype: int64>

In [5]:
# Создаем признаки принадлежности к классу
def set_class(row):
    row[row['Class 1']] = True
    row[row['Class 2']] = True
    return row

pokemon = pokemon.apply(set_class, axis=1)

In [6]:
pokemon[pokemon.columns.difference(['Name', 'Class 2'])] = pokemon[pokemon.columns.difference(['Name', 'Class 2'])].fillna(False)
pokemon

,Attack,Class 1,Class 2,Defense,Fire,Grass,HP,Legendary,Name,Poison,...,Electric,Ground,Fairy,Fighting,Psychic,Rock,Steel,Ice,Ghost,Dark
0,49,Grass,Poison,49,False,True,45,False,Bulbasaur,True,...,False,False,False,False,False,False,False,False,False,False
1,62,Grass,Poison,63,False,True,60,False,Ivysaur,True,...,False,False,False,False,False,False,False,False,False,False
2,82,Grass,Poison,83,False,True,80,False,Venusaur,True,...,False,False,False,False,False,False,False,False,False,False
3,100,Grass,Poison,123,False,True,80,False,Mega Venusaur,True,...,False,False,False,False,False,False,False,False,False,False
4,52,Fire,NaN,43,True,False,39,False,Charmander,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,100,Rock,Fairy,150,False,False,50,True,Diancie,False,...,False,False,True,False,False,True,False,False,False,False
796,160,Rock,Fairy,110,False,False,50,True,Mega Diancie,False,...,False,False,True,False,False,True,False,False,False,False
797,110,Psychic,Ghost,60,False,False,80,True,Hoopa Confined,False,...,False,False,False,False,True,False,False,False,True,False
798,160,Psychic,Dark,60,False,False,80,True,Hoopa Unbound,False,...,False,False,False,False,True,False,False,False,False,True


In [7]:
def f_test(group1, group2):
    f = np.var(group1, ddof=1)/np.var(group2, ddof=1)
    nun = group1.size-1
    dun = group2.size-1
    p_value = 1-st.f.cdf(f, nun, dun)
    return f, p_value

In [8]:
p1_df = pokemon.loc[~(pokemon['Grass'] & pokemon['Rock'])]

In [10]:
stat, p = f_test(
    p1_df.loc[p1_df['Grass'], 'Attack'], 
    p1_df.loc[p1_df['Rock'], 'Attack']
    )
stat, p

(0.48262460954678854, 0.9990088840898671)

In [42]:
stat, p = st.ttest_ind(
    p1_df.loc[p1_df['Grass'], 'Attack'], 
    p1_df.loc[p1_df['Rock'], 'Attack']
)
stat, p

(-3.729457582692065, 0.00027333381459902664)

In [12]:
print(
    p1_df.loc[p1_df['Grass'], 'Attack'].mean(),
    p1_df.loc[p1_df['Rock'], 'Attack'].mean()    
)


73.73118279569893 91.78571428571429


Отклоняем гипотезу о равенстве средних. Покемоны класса Rock атакуют сильней

<div class="alert alert-info">
<b>Задание № 2:</b>
    
Профессор Оук уже долго не может спать по ночам, ведь его волнует вопрос, а правда ли, что покемоны в классе `Water` в среднем быстрее, чем покемоны в классе `Normal`.
    
    
Проверьте, прав ли он, и убедите его в своём выводе статистически.
    
Примечание: если есть покемоны, которые относятся к обоим классам, выбросьте их;
    
Вы можете предположить, что распределение скорости движения является нормальным для всех классов покемонов.
</div>

In [13]:
p2_df = pokemon.loc[~(pokemon['Water'] & pokemon['Normal'])]

In [44]:
stat, p = f_test(
    p2_df.loc[p2_df['Water'], 'Speed'], 
    p2_df.loc[p2_df['Normal'], 'Speed']
    )
stat, p

(0.6403955333179285, 0.9907447713433752)

In [45]:
stat, p = st.ttest_ind(
    p2_df.loc[p2_df['Water'], 'Speed'], 
    p2_df.loc[p2_df['Normal'], 'Speed']
)
stat, p

(-2.150913330007121, 0.032553535513910326)

Нет различий между скоростью покемонов классов Water и Normal

<div class="alert alert-info">
<b>Задание № 3:</b>
    
Профессор Оук тот еще безумец. Он изобрёл сыворотку, способную ускорить покемона. Однако мы усомнились в эффективности его вакцины. Професоор дал эту сыворотку следующим покемонам: смотри массив `treathed_pokemon`. Проверьте, работает ли вообще его сыворотка, убедите всех в своём выводе статистически.
    
    
Вы можете предположить, что распределение скорости движения является нормальным для всех классов покемонов.

</div>

In [16]:
# Покемоны, которые принимали сыворотку увеличения скорости
treathed_pokemon = ['Mega Beedrill', 'Mega Alakazam',
                    'Deoxys Normal Forme', 'Mega Lopunny']

Логика моих размышлений такая. Если не прав, дайте обратную связь, пожалуйста.
Мы сравниваем скорость испытуемого покемона со средней скоростью по всему его классу.
Если есть отличия по обоим, то сыворотка действует. Если не отличается хотя бы по одному, то нет
Для сравнения используем t-тест для одной выборки

In [41]:
for cur_p in treathed_pokemon:
    speed = pokemon.loc[pokemon['Name'] == cur_p, 'Speed'].values[0]
    class1 = pokemon.loc[pokemon['Name'] == cur_p, 'Class 1'].values[0]
    class2 = pokemon.loc[pokemon['Name'] == cur_p, 'Class 2'].values[0]
    _, pval1 = st.ttest_1samp(
        pokemon.loc[pokemon[class1], 'Speed'],
        speed
        )
    if class2 is not None:
         _, pval2 = st.ttest_1samp(
            pokemon.loc[pokemon[class2], 'Speed'],
            speed
        )
    else:
        pval2 = None
        
    print(
        f'Name: {cur_p}\t Class 1: {class1}\t p-value 1: {pval1:.5f}\t'
        f'Class 2: {class2}\t p-value2: {pval2:.5f}'
        )

Name: Mega Beedrill	 Class 1: Bug	 p-value 1: 0.00000	Class 2: Poison	 p-value2: 0.00000
Name: Mega Alakazam	 Class 1: Psychic	 p-value 1: 0.00000	Class 2: nan	 p-value2: 0.00000
Name: Deoxys Normal Forme	 Class 1: Psychic	 p-value 1: 0.00000	Class 2: nan	 p-value2: 0.00000
Name: Mega Lopunny	 Class 1: Normal	 p-value 1: 0.00000	Class 2: Fighting	 p-value2: 0.00000


Отклоняем все нулевые гипотезы. Сыворотка работает

<div class="alert alert-info">
<b>Задание № 4:</b>
    
Профессор Оук всегда любил истории про легендарных покемонов. Однако профессор не очень уверен, что они лучше остальных покемонов. Оук предложил разобраться в этом нам. Проверьте, действительно ли сумма характеристик `HP`,`Attack`,`Defense` у легендарных покемонов выше, чем у других покемонов?

А произведение этих же параметров?

Найдите ответы на эти вопросы и убедите всех в своём выводе статистически.
   

Вы можете предположить, что распределение сум и произведений этих параметров является нормальным для всех классов покемонов.

</div>

pokemon['Sum'] = pokemon['HP'] + pokemon['Attack'] + pokemon['Defense']
pokemon['Mul'] = pokemon['HP'] * pokemon['Attack'] * pokemon['Defense']

In [47]:
stat, p = st.ttest_ind(
    pokemon.loc[pokemon['Legendary'], 'Sum'],
    pokemon.loc[~pokemon['Legendary'], 'Sum']
)
stat, p

(11.591852366462316, 7.970942205722087e-29)

In [48]:
stat, p = st.ttest_ind(
    pokemon.loc[pokemon['Legendary'], 'Mul'],
    pokemon.loc[~pokemon['Legendary'], 'Mul']
)
stat, p

(13.263253408231844, 1.992664308842282e-36)

In [51]:
pokemon[['Sum', 'Mul', 'Legendary']].groupby('Legendary').mean()

,Sum,Mul
Legendary,,
False,214.410884,4.250414e+05
True,309.076923,1.085942e+06


И сумма и произведение параметров у легендарных покемонов выше

<div class="alert alert-info">
<b>Задание № 5:</b>
    
Профессор Оук частенько наблюдает за боями покемонов. После очередных таких боёв Оук выделил четыре класса `best_defence_class`, которые на его взгляд одинаковы по "силе обычной защиты" `Defense`.

Проверьте, действительно ли эти классы покемонов не отличаются по уровню защиты статистически значимо? Всё та же статистика вам в помощь!
   

Вы можете предположить, что распределение параметров защитных характеристик является нормальным для всех классов покемонов.

</div>

In [52]:
best_defence_class = ['Rock', 'Ground', 'Steel', 'Ice']
best_defence_class

['Rock', 'Ground', 'Steel', 'Ice']

In [60]:
p_best = [pokemon.loc[pokemon[class_], 'Defense'] for class_ in best_defence_class]
p_best

[80     100
 81     115
 82     130
 103    160
 119     95
 120    120
 149    100
 150    125
 151     90
 152    105
 153     65
 154     85
 200    115
 230    230
 237    120
 240     85
 265     50
 266     70
 267    110
 268    150
 323    135
 330    100
 331    140
 332    180
 369     65
 370     85
 377     77
 378     97
 379     50
 380    100
 404    130
 414    200
 453     40
 454     60
 455    118
 456    168
 486     95
 515    130
 528    145
 583     85
 584    105
 585    130
 618     85
 619    125
 625    103
 626    133
 627     45
 628     65
 700     90
 758     67
 759    115
 766     77
 767    119
 768     50
 769     72
 773    150
 795    150
 796    110
 Name: Defense, dtype: int64,
 32      85
 33     110
 36      87
 39      77
 55      25
       ... 
 684     80
 708     90
 709     90
 728     77
 794    121
 Name: Defense, Length: 67, dtype: int64,
 88      70
 89      95
 220    140
 223    200
 224    230
 228    100
 229    140
 245    140
 328

In [61]:
stat, p = st.f_oneway(*p_best)
stat, p

(10.82012408304731, 1.231528168152447e-06)

Классы отличаются по уровню защиты

# **Примечание:**

Домашнее задание сдается ссылкой [Google Colab](https://colab.research.google.com/). Мы не сможем проверить его или помочь, если вы пришлете:

*   файлы;
*   архивы;
*   скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в Discord.

**Как правильно задавать вопросы аспирантам, преподавателям и коллегам:**

Прежде чем задать вопрос, попробуйте найти ответ в интернете. Навык самостоятельного поиска информации — один из важнейших. Каждый практикующий специалист любого уровня делает это ежедневно.

Сформулируйте вопрос по алгоритму:

1.   Что я делаю?
2.   Какого результата я ожидаю?
3.   Как фактический результат отличается от ожидаемого?
4.   Что я уже попробовал сделать, чтобы исправить проблему?

По возможности прикрепите к вопросу скриншоты либо ссылки на код. Не выкладывайте все решение, оставляйте только проблемный и воспроизводимый участок кода.